# 22. Generate parenthesis
---

## 问题描述
---
Given n pairs of parentheses, write a function to generate all combinations of well-formed parentheses.
```
For example, given n = 3, a solution set is:

[
  "((()))",
  "(()())",
  "(())()",
  "()(())",
  "()()()"
]
```

## 思路
---
看到这个问题第一反应就是想到刚在《算法导论》中看到的“矩阵链乘法问题”，这道题和计算n个矩阵的括号化方案非常类似。对于$n$对括号，令$P(n)$等于所有括号方案，可得到如下递归公式,其中left和right分别表示```'('```和```')'```：
> \begin{equation}
P(n)=\left\{
\begin{aligned}
None, && n=0 \\
\sum_{i=0}^{n-1}[left+P(i)+right+P(n-i-1)], && n>0
\end{aligned}
\right.
\end{equation}

通过求解递归公式，可知括号方案的个数与$n$呈指数关系（$\Omega(2^n)$）。
> 从矩阵链乘法方法的解法得到启发，可知这个题同样可通过动态规划来求解。因为从上述公式可知，对每一个问题的解都只依赖于“更小的”子问题的解，而子问题又只依赖于“更更小的”子子问题的解，最终退化为平庸解。所以我们可以按照子问题规模，从小到大顺序求解。求解$P(n)$的时候，它所依赖的$P(i), i = 0,1,2,..., n-1$个问题都已经解决，结果已经保存。

## 代码如下：
---

In [13]:
def generateParentheses(n):
    if not n: return ''
    result = [[] for _ in range(n+1)]
    result[0].append('')
    for i in range(1,n+1):
        for j in range(i):
            result[i] += ['('+x+')'+y for x in result[j]\
                          for y in result[i-j-1]]
    return result[-1]

## 测试结果：
---
由于括号化方案的数目随$n$指数增长，所以，我们这里只看下其方案的数目而不打印所以的括号化方案 （不然整版全是括号了。。。）。

In [14]:
for i in range(10):
    print('n = ',i,':', len(generateParentheses(i)))

n =  0 : 0
n =  1 : 1
n =  2 : 2
n =  3 : 5
n =  4 : 14
n =  5 : 42
n =  6 : 132
n =  7 : 429
n =  8 : 1430
n =  9 : 4862


## 再谈一下动态规划
---
之前在Coursera上算法课的时候，老师最开始就讲了归并排序，当时还奇怪效率高的算法不应该留到最后讲么？最近翻《算法导论》也发现归并排序也是在很开始就讲了，而目的就是介绍一个很重要的思想“divide and conquer”，而后面看动态规划的时候，感觉其核心也是divide and conquer。也就是：当拿到一个问题，发现，哇好复杂！咋整呢？划分为几个更简单的问题来求解。就如同那个笑话：

甲：发现邻居家房子着火了怎么办？

乙：叫消防车来灭火。

甲：那发现邻居家来贼了怎么办？

乙：点燃他家房子。这样就变为我熟悉的问题了。

大抵是这个意思吧。

> 举一个典型的例子，求解斐波那契数列。根据其定义，每一项是前两项的和，直观上我们可以写出如下的递归程序：

In [15]:
def fib1(n):
    f = 0;
    g = 1;
    if n==0:
        return f;
    if n==1:
        return g
    return fib1(n-1)+fib1(n-2)

这个程序虽然简单，但也存在很明显的问题。比如我们求解$fib1(5)$的时候，会去调用$fib1(4)$和f$ib1(3)$，而求解$fib1(4)$的时候又会调用$fib1(3)$和$fib1(2)$，这样就有很多重复调用的地方，也就是说，同一个问题解决了很多次，这是完全没有必要的。在问题规模很大的时候，所需时间会达到天文数字。事实上，这种方法的计算量也是随$n$指数增长的，当$n=50$的时候，就基本在短时间内等不出结果了。

### 备忘机制
---
那么，怎么避免重复解决的子问题呢？一个直观的解决办法就是，将已经解决的子问题备忘，这样每次调用子问题的时候，我们先检查备忘，看这个问题有没有被解决过，如果有，那么我们直接拿来用；如果没有，我们就解决了并将其备忘，留待下次使用。

In [16]:
def fib2(n):
    f = [-1 for i in range(n+1)]
    return fib_mem(n,f)

def fib_mem(n, f):
    if f[n]>=0:
        return f[n]
    elif n==0:
        f[0] = 0
    elif n==1:
        f[1] = 1
    else:
        for j in range(2,n+1):
            f[n]=f[n-1]+f[n-2]
    return f[n]

### 自底向上解决
---
和前面的题目类似，这个问题也可以从最基本的问题开始，自底向上顺序求解。代码如下：

In [17]:
def fib3(n):
    f = 0;
    g = 1;
    if n==0:
        return f;
    if n==1:
        return g
    while(n-1):
        f,g = g, f+g
        n=n-1
    return g

### 效率比较
---
我们可以用python的time模块对上述三种算法的效率进行测试。由于第一种方法的运行时间是指数级的，所以参数$n$在大于50之后所需时间开销已经非常大，所以我们在计算 fib1(n)时取 n=30，而计算 $fib2(n)$ 和 $fib3(n)$ 时取$n=5000$。可以看出，方法二和方法三能对很大的 $n$ 进行明显快速的计算，达到线性的时间复杂度 $O(n)$。且自底向上的方法的时间开销会更小。

In [9]:
import time
start = time.clock()
for i in range(30):
    fib1(i)
t1 = time.clock()-start
start = time.clock()
for i in range(5000):
    fib2(i)
t2 = time.clock()-start
start = time.clock()
for i in range(5000):
    fib3(i)
t3 = time.clock()-start
print('t1=',t1,'s')
print('t2=',t2,'s')
print('t3=',t3,'s')

t1= 1.1731497510719984 s
t2= 4.520209350780285 s
t3= 5.867298975132483 s


## 小结
---

想起好多